# Adaptive RAG system

In traditional RAG systems, retrieval is often a generic step — every query is handled with the same strategy regardless of its nature. But not all queries are the same. Some require precision, others benefit from multiple perspectives or personalized context. This notebook presents an Adaptive RAG system that intelligently selects the most suitable retrieval strategy for a given query type, using LLMs both for classification and context enhancement.

The goal is to build a pipeline that automatically classifies the type of query (e.g., factual, analytical, opinion-based, or contextual) and applies a retrieval strategy that best matches the needs of that query. The retrieved results are then used by a language model to generate accurate and nuanced answers.

In [1]:
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS

from langchain_core.retrievers import BaseRetriever
from typing import Dict, Any, List
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Query classification
To choose the best retrieval strategy for a given user query, we first need to understand the intent behind that query. Is the user asking for a factual piece of information, trying to analyze a concept, exploring opinions, or expecting an answer tailored to their specific context?

This step is critical. Instead of relying on hardcoded rules or heuristics, we leverage an LLM to classify the query based on its semantics. The classifier returns one of four high-level categories:
- Factual – Looking for specific, verifiable information.
- Analytical – Seeks an explanation or breakdown, often multi-faceted.
- Opinion – Asks for subjective perspectives or diverse viewpoints.
- Contextual – Relies on user context or environment to interpret the query.

This enables the system to adaptively route the query to the most appropriate strategy in the next step.

Let's define the `QueryClassifier` class, which uses a prompt to guide the LLM in making this decision. We will also define a simple Pydantic model to structure the LLM output.

In [2]:
# Define the schema for the classifier's output using Pydantic.
# This ensures the LLM returns a result in a structured format, specifically one of our four categories.
class categories_options(BaseModel):
        category: str = Field(description="The category of the query, the options are: Factual, Analytical, Opinion, or Contextual", example="Factual")


# Class that wraps the classification chain
class QueryClassifier:
    def __init__(self):
        # Initialize the LLM with zero temperature for deterministic output.
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18", max_tokens=4000)
        # Define the prompt template used to classify a query.
        self.prompt = PromptTemplate(
            input_variables=["query"],
            template="Classify the following query into one of these categories: Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nCategory:"
        )
        # Chain the prompt with the LLM and define the output schema. This ensures we always get a well-structured response that conforms to the expected format.
        self.chain = self.prompt | self.llm.with_structured_output(categories_options)

    # Define the classify method to run the query through the chain and extract the predicted category.
    def classify(self, query):
        print("\n\nclasiffying query")
        return self.chain.invoke(query).category

This component uses LLM to determine which strategy to follow next. It returns one of the predefined categories. This `QueryClassifier` uses a few-shot LLM prompt and expects structured output. The result is a single word: the query’s category.
- The `PromptTemplate` defines how we instruct the LLM to perform the classification. It keeps the task clear and narrow.
- `ChatOpenAI` is initialized with zero temperature, which makes the model output consistent and non-random—crucial for classification.
- The combination of `PromptTemplate` and `ChatOpenAI` is piped (`|`) together with a structured output wrapper (`with_structured_output`) that parses the LLM output into a well-defined Pydantic model (`categories_options`).
- Finally, `self.chain.invoke(query)` runs the query through the prompt and LLM, and parses the output to get the predicted category.

### Base retrieval engine
Before we define the specialized retrieval strategies, we first create a shared foundation for them to build upon.

This `BaseRetrievalStrategy` class encapsulates all the common logic for setting up retrieval: breaking down long documents into smaller chunks, embedding those chunks into a high-dimensional vector space, and storing them in a fast vector index (FAISS). It also instantiates an LLM, which child classes can reuse to enhance or rank the retrieved results. All strategies will inherit from a shared retrieval logic.

In [3]:
# Base retrieval engine from which all specialized strategies will inherit
class BaseRetrievalStrategy:
    def __init__(self, texts):
        # Initialize an embedding model from OpenAI to vectorize text
        self.embeddings = OpenAIEmbeddings()
        # Split long documents into smaller chunks for better retrieval performance
        text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
        self.documents = text_splitter.create_documents(texts)

        # Build the FAISS vector store from the document embeddings
        self.db = FAISS.from_documents(self.documents, self.embeddings)
        # Initialize the LLM once for all retrieval tasks
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18", max_tokens=4000)

    # Basic retrieve method that searches for top-k most similar documents
    def retrieve(self, query, k=4):
        return self.db.similarity_search(query, k=k)

Here, we are setting up the core building blocks needed for all retrieval strategies.
- **Embeddings**: We convert all the input text into numerical vector representations using `OpenAIEmbeddings`2. These vectors capture semantic meaning, so similar texts end up close to each other in the vector space.
- **Chunking**: Since large documents can be too long for embedding models or miss fine-grained details, we split the text into manageable 800-character chunks with no overlap. This gives better resolution during retrieval.
- **Vector store**: We store these vectors using FAISS, which is an efficient similarity search engine. It allows fast lookup of the most relevant chunks based on cosine similarity.
- **LLM**: We initialize a GPT-4o model so that any child class can use it to enhance queries, rank results, or do more sophisticated logic.

The `retrieve` method here is a simple similarity search. Child strategies (like Factual, Analytical, etc.) can override this method to plug in more advanced behavior like query expansion, document reranking, sub-query generation, etc.

### Strategy 1: Factual retrieval
This strategy is designed for queries that expect concrete, verifiable information—things like dates, definitions, or specific facts. To improve precision, it enhances the user query before performing the search, then uses the LLM again to rank results based on how relevant they are.

Compared to a basic vector search, this method injects more semantic understanding into the process—both in how we frame the query and how we interpret the results. It’s slower than raw similarity search but produces much more accurate answers when factual precision matters.

In [4]:
# Schema used to capture the LLM-generated relevance score for a document
class relevant_score(BaseModel):
        score: float = Field(description="The relevance score of the document to the query", example=8.0)

# Factual retrieval strategy that builds on the BaseRetrievalStrategy
class FactualRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=4):
        print("retrieving factual")
        # Use LLM to enhance the user's query for better matching
        enhanced_query_prompt = PromptTemplate(
            input_variables=["query"],
            template="Enhance this factual query for better information retrieval: {query}"
        )
        query_chain = enhanced_query_prompt | self.llm
        enhanced_query = query_chain.invoke(query).content
        print(f'enhande query: {enhanced_query}')

        # Perform similarity search using the enhanced query and over-fetch results
        docs = self.db.similarity_search(enhanced_query, k=k*2)

        # Use LLM to rank the relevance of each retrieved document (1–10)
        ranking_prompt = PromptTemplate(
            input_variables=["query", "doc"],
            template="On a scale of 1-10, how relevant is this document to the query: '{query}'?\nDocument: {doc}\nRelevance score:"
        )
        ranking_chain = ranking_prompt | self.llm.with_structured_output(relevant_score)

        ranked_docs = []
        print("ranking docs")
        for doc in docs:
            input_data = {"query": enhanced_query, "doc": doc.page_content}
            # Ask the LLM to assign a relevance score for each document
            score = float(ranking_chain.invoke(input_data).score)
            ranked_docs.append((doc, score))

        # Sort documents by score in descending order and return top-k
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:k]]

This strategy improves the factual retrieval by refining the question and only returning documents with the highest LLM-judged relevance.
- It first enhances the query using the LLM. This helps bridge the gap between user phrasing and the language used in the documents. For example, “When was Tesla founded?” might become “Provide the founding year of the Tesla company.”
- Then it retrieves twice as many documents as needed—this is just to give the ranking step more material to work with.
- The next key step is ranking. The LLM is asked to evaluate how relevant each document is on a scale from 1 to 10. This gives us a more intelligent filter than relying solely on embedding similarity.
- After ranking, it sorts and slices the top `k` documents to return only the most relevant ones.

It’s especially useful in knowledge-intensive domains like legal, scientific, or technical documentation.

### Strategy 2: Analytical retrieval
Analytical questions usually don’t have a single, straightforward answer. Instead, they require combining insights from multiple angles. For instance, a question like “How successful is New York as a global city?” needs economic data, demographic context, infrastructure, maybe even historical growth patterns.

This strategy addresses that by breaking the question into smaller sub-questions, fetching documents for each one, and finally curating a diverse set of sources using the LLM.

In [5]:
# Schema for selecting document indices based on diversity and relevance
class SelectedIndices(BaseModel):
    indices: List[int] = Field(description="Indices of selected documents", example=[0, 1, 2, 3])

# Schema for the LLM to return multiple sub-queries
class SubQueries(BaseModel):
    sub_queries: List[str] = Field(description="List of sub-queries for comprehensive analysis", example=["What is the population of New York?", "What is the GDP of New York?"])

# Analytical retrieval strategy based on sub-query decomposition and diversity scoring
class AnalyticalRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=4):
        print("retrieving analytical")

        # Step 1: Generate k sub-questions that explore the topic more fully
        sub_queries_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Generate {k} sub-questions for: {query}"
        )
        llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18", max_tokens=4000)
        sub_queries_chain = sub_queries_prompt | llm.with_structured_output(SubQueries)

        input_data = {"query": query, "k": k}
        sub_queries = sub_queries_chain.invoke(input_data).sub_queries
        print(f'sub queries for comprehensive analysis: {sub_queries}')

        # Step 2: Perform similarity search for each sub-query (over-fetching to widen scope)
        all_docs = []
        for sub_query in sub_queries:
            all_docs.extend(self.db.similarity_search(sub_query, k=2))

        # Step 3: Use LLM to select a final set of k diverse and relevant documents
        diversity_prompt = PromptTemplate(
            input_variables=["query", "docs", "k"],
            template="""Select the most diverse and relevant set of {k} documents for the query: '{query}'\nDocuments: {docs}\n
            Return only the indices of selected documents as a list of integers."""
        )
        diversity_chain = diversity_prompt | self.llm.with_structured_output(SelectedIndices)

        # Convert docs into a preview format so the LLM can choose among them
        docs_text = "\n".join([f"{i}: {doc.page_content[:50]}..." for i, doc in enumerate(all_docs)])
        # Format the documents into a brief numbered list (truncated for prompt length)
        input_data = {"query": query, "docs": docs_text, "k": k}

        # Step 4: Let the LLM choose the final k documents by index
        selected_indices_result = diversity_chain.invoke(input_data).indices
        print(f'selected diverse and relevant documents')

        # Step 5: Return only the selected documents, with index bounds safety
        return [all_docs[i] for i in selected_indices_result if i < len(all_docs)]

This strategy relies heavily on decomposition and diversification. Instead of trying to answer an open-ended or multifaceted question head-on, it breaks the problem into smaller, more concrete questions. These sub-questions are generated by the LLM and treated as independent search prompts.

It then searches the vector DB multiple times—once for each sub-query—to build a pool of potentially useful documents. Of course, some of these documents will overlap or be redundant. That’s where the final LLM step comes in.

The model is asked to pick a balanced and diverse subset from the entire result pool. Instead of filtering based only on similarity, this lets the system ensure that the final selection includes multiple perspectives or types of evidence.

The result is a curated mix of documents that represent different dimensions of the original question—ideal for synthesis, comparison, or in-depth reasoning.

### Strategy 3: Opinion retrieval
Opinion-based queries don’t rely on a single source of truth. Instead, they benefit from diversity—different angles, personal stances, or even conflicting views. This strategy uses the LLM to first identify possible viewpoints, then curates a representative selection of documents aligned with those views.

In [6]:
# Strategy for retrieving diverse opinions on subjective or open-ended topics
class OpinionRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=3):
        print("retrieving opinion")
        # Step 1: Use the LLM to extract k potential viewpoints on the given topic
        viewpoints_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Identify {k} distinct viewpoints or perspectives on the topic: {query}"
        )
        viewpoints_chain = viewpoints_prompt | self.llm
        input_data = {"query": query, "k": k}
        # The output is expected to be a string of viewpoints separated by newlines
        viewpoints = viewpoints_chain.invoke(input_data).content.split('\n')
        print(f'viewpoints: {viewpoints}')

        # Step 2: Use similarity search to retrieve documents for each viewpoint
        all_docs = []
        for viewpoint in viewpoints:
            # Boost relevance by combining original query with viewpoint context
            all_docs.extend(self.db.similarity_search(f"{query} {viewpoint}", k=2))

        # Step 3: Use LLM to classify and select the most diverse and representative opinions
        opinion_prompt = PromptTemplate(
            input_variables=["query", "docs", "k"],
            template="Classify these documents into distinct opinions on '{query}' and select the {k} most representative and diverse viewpoints:\nDocuments: {docs}\nSelected indices:"
        )
        opinion_chain = opinion_prompt | self.llm.with_structured_output(SelectedIndices)

        # Format the docs into an indexed summary to help the LLM make selection decisions
        docs_text = "\n".join([f"{i}: {doc.page_content[:100]}..." for i, doc in enumerate(all_docs)])
        input_data = {"query": query, "docs": docs_text, "k": k}

        # Step 4: Let the LLM choose the final k documents by index
        selected_indices = opinion_chain.invoke(input_data).indices
        print(f'selected diverse and relevant documents')

        # Step 5: Return the selected subset of documents
        return [all_docs[int(i)] for i in selected_indices.split() if i.isdigit() and int(i) < len(all_docs)]

This strategy is tuned for subjective, debatable, or personal-opinion questions like “Is remote work good for productivity?” or “Should AI be regulated more strictly?” These don’t have a correct answer—they invite a variety of stances.

The first move is to ask the LLM for a few distinct viewpoints. Think of this like surfacing the axes of debate. These might include political, cultural, ethical, or practical perspectives depending on the query.

Each viewpoint is then used to anchor a separate similarity search, widening the scope and ensuring that multiple angles are considered—not just the most dominant or popular one.

After that, the LLM acts as a selector and classifier. It reviews a pool of candidate documents and identifies the most representative and diverse ones, filtering out redundancy or bias. This is especially useful in avoiding echo chambers or one-sided responses.

Finally, we return the best-matching documents—each ideally aligned with a unique opinion.

This retrieval path is especially valuable for generating balanced summaries, debate-style content, or constructing contrasting points of view for synthesis.

### Strategy 4: Contextual retrieval
When a user’s query depends on additional context—like their role, goals, preferences, or prior actions—retrieving relevant information requires more than just matching keywords. This strategy uses the LLM to first reinterpret the query through the lens of that context, and then performs a context-aware ranking of the documents it finds.

In [7]:
# Strategy for handling queries that depend on user context or prior information
class ContextualRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=4, user_context=None):
        print("retrieving contextual")
        # Step 1: Use LLM to incorporate user context into the query
        context_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template="Given the user context: {context}\nReformulate the query to best address the user's needs: {query}"
        )
        context_chain = context_prompt | self.llm
        # If no context is provided, use a fallback message
        input_data = {"query": query, "context": user_context or "No specific context provided"}
        # The LLM returns a rewritten query that better reflects the user's intent
        contextualized_query = context_chain.invoke(input_data).content
        print(f'contextualized query: {contextualized_query}')

        # Step 2: Perform a similarity search using the rewritten contextualized query. We retrieve more documents initially to allow room for intelligent filtering
        docs = self.db.similarity_search(contextualized_query, k=k*2)

        # Step 3: Use LLM to rank the relevance of retrieved documents considering the query and user context
        ranking_prompt = PromptTemplate(
            input_variables=["query", "context", "doc"],
            template="Given the query: '{query}' and user context: '{context}', rate the relevance of this document on a scale of 1-10:\nDocument: {doc}\nRelevance score:"
        )
        ranking_chain = ranking_prompt | self.llm.with_structured_output(relevant_score)
        print("ranking docs")

        ranked_docs = []
        for doc in docs:
            input_data = {"query": contextualized_query, "context": user_context or "No specific context provided", "doc": doc.page_content}
            # Parse the LLM’s relevance rating and pair it with the doc
            score = float(ranking_chain.invoke(input_data).score)
            ranked_docs.append((doc, score))


        # Step 4: Sort documents by relevance and return the top results
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:k]]

This strategy is all about personalization. It assumes that the same query—say, "What should I read next?"—could mean totally different things depending on who’s asking. Maybe the user is a software engineer looking for technical papers, or a student exploring philosophy.

First, the LLM reformulates the query using the provided context. This step is crucial because embedding-based retrieval systems are very literal. If the query doesn't fully express the user's need, the retrieved results can miss the mark.

After retrieving a wide set of candidate documents, we call on the LLM again—this time to rank them. But it doesn't just consider the textual overlap. Instead, it interprets how well each document fits the user’s context, giving us a much more tailored set of results.

This approach is useful in scenarios like:
- Personalized assistants that adjust to user goals or preferences
- Multi-turn dialogue systems with conversational memory
- Enterprise retrieval where user role and intent matter

Contextual retrieval is essentially contextualization both ways—first in how the query is phrased, and second in how relevance is judged.

### Putting it all together: Adaptive retriever
This component is the brain of the retrieval system. It acts like an intelligent router: it first figures out what kind of question it's dealing with, then dynamically picks the most appropriate retrieval strategy.

In [8]:
# Central orchestrator that selects the best retrieval strategy based on query type
class AdaptiveRetriever:
    def __init__(self, texts: List[str]):
        # Load the query classifier that will label incoming queries
        self.classifier = QueryClassifier()
        # Preload all strategies with the same knowledge base (texts)
        self.strategies = {
            "Factual": FactualRetrievalStrategy(texts),
            "Analytical": AnalyticalRetrievalStrategy(texts),
            "Opinion": OpinionRetrievalStrategy(texts),
            "Contextual": ContextualRetrievalStrategy(texts)
        }

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Step 1: Determine query type using LLM-based classifier
        category = self.classifier.classify(query)
        # Step 2: Pick and run the matching retrieval strategy
        strategy = self.strategies[category]
        return strategy.retrieve(query)

This `AdaptiveRetriever` acts like a switchboard operator. When a user sends in a query, it doesn’t just throw it into a vector search blindly—it first asks the classifier to label the question as one of four types: Factual, Analytical, Opinion, or Contextual. That classification is based on the semantics of the query itself, using an LLM fine-tuned to recognize intent.

Then, the system selects the appropriate retrieval strategy accordingly. Each strategy knows how to handle its specific case—whether it needs to reformulate the query, decompose it, or consider user context. The `AdaptiveRetriever` lets us combine all these approaches in a modular, extensible way.

### Integrating with LangChain’s `BaseRetriever`

To make our adaptive retriever compatible with LangChain’s tools and agents, we wrap it in a class that inherits from LangChain’s BaseRetriever interface. This gives us compatibility with chains, agents, and any LangChain component that expects a retriever.

In [9]:
# LangChain-compatible wrapper around our custom adaptive retriever
class PydanticAdaptiveRetriever(BaseRetriever):
    # Embed the AdaptiveRetriever as an internal field
    adaptive_retriever: AdaptiveRetriever = Field(exclude=True)

    class Config:
        # Allow non-pydantic objects like AdaptiveRetriever to be used inside this class
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Required sync interface for LangChain retrievers
        return self.adaptive_retriever.get_relevant_documents(query)

    # Optional async version for compatibility with async chains/agents
    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        # For now, we reuse the sync method; we can parallelize internally if needed
        return self.get_relevant_documents(query)

This class is a bridge between our custom logic and LangChain’s standardized retriever ecosystem. LangChain expects any retriever to implement the `get_relevant_documents` method—and optionally, the async version as well. So here, we wrap your `AdaptiveRetriever` in a LangChain-friendly shell by subclassing `BaseRetriever`.

The `adaptive_retriever` field is excluded from Pydantic serialization (e.g., when storing metadata or configurations) since it's not a standard serializable field—it holds a complex object. The `Config` subclass tells Pydantic to allow such non-standard types, which avoids validation issues.

Ultimately, this small wrapper unlocks full interoperability with LangChain pipelines, memory modules, and tools. We can now plug this into an agent, use it in a retriever chain, or swap it in wherever LangChain expects a retriever. It’s the final step that turns our custom logic into a fully reusable component.

### End-to-end adaptive RAG pipeline
This class orchestrates the entire retrieval-and-generation process. It decides how to search, what to retrieve, and how to turn that into a coherent answer. It’s the complete RAG system—from raw query to final response.

In [10]:
class AdaptiveRAG:
    def __init__(self, texts: List[str]):
        # Initialize the adaptive retriever pipeline with input documents
        adaptive_retriever = AdaptiveRetriever(texts)

        # Wrap it in a LangChain-compatible retriever
        self.retriever = PydanticAdaptiveRetriever(adaptive_retriever=adaptive_retriever)

        # Instantiate the LLM for final answer generation
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18", max_tokens=4000)

        # Define the prompt template used to guide answer generation
        prompt_template = """Use the following pieces of context to answer the question at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.

        {context}

        Question: {question}
        Answer:"""

        # Wrap the prompt with LangChain's PromptTemplate
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        # Create a simple LLM chain: prompt -> LLM
        self.llm_chain = prompt | self.llm


    def answer(self, query: str) -> str:
        # Step 1: Retrieve the most relevant documents for the query
        docs = self.retriever.invoke(query)

        # Step 2: Prepare the context for the LLM by joining the retrieved texts
        input_data = {"context": "\n".join([doc.page_content for doc in docs]), "question": query}

        # Step 3: Invoke the LLM with the context-rich prompt to generate the answer
        return self.llm_chain.invoke(input_data)

This class is the top-level orchestrator—the part that a user or agent interacts with directly. When a query comes in, it sends it to the adaptive retriever, which figures out what type of question it is (factual, analytical, opinion, or contextual), and applies the right strategy to gather relevant documents.

Once the context is retrieved, it’s passed to the LLM using a clearly defined prompt that emphasizes factuality and avoids hallucination. The LLM then generates a final answer using this filtered, context-aware input.

The result is a fully adaptive, query-aware RAG pipeline that can intelligently handle a wide variety of question types without any manual intervention.

### Test run: Showcase the adaptive system with different types of queries
This section demonstrates how the adaptive retrieval system dynamically selects the right strategy based on the nature of the input question.

In [14]:
# Sample document collection (you can expand this with more data)
texts = [
    "The Earth is the third planet from the Sun and the only astronomical object known to harbor life."
    ]

# Instantiate the Adaptive RAG system
rag_system = AdaptiveRAG(texts)

# Test 1: A factual question – should trigger the FactualRetrievalStrategy
factual_result = rag_system.answer("What is the distance between the Earth and the Sun?").content
print(f"Answer: {factual_result}")

# Test 2: An analytical question – should break the query into sub-questions
analytical_result = rag_system.answer("How does the Earth's distance from the Sun affect its climate?").content
print(f"Answer: {analytical_result}")

# Test 3: An opinion-based question – should collect diverse perspectives
opinion_result = rag_system.answer("What are some controversial beliefs scientists hold about how life began on Earth?").content
print(f"Answer: {opinion_result}")

# Test 4: A context-dependent question – simulates a contextual reformulation
contextual_result = rag_system.answer("In the context of designing life-supporting environments, what does Earth’s place in the Solar System teach us?").content
print(f"Answer: {contextual_result}")



clasiffying query
retrieving factual
enhande query: To improve the specificity and depth of your query, you might consider rephrasing it as follows:

"What is the average distance between the Earth and the Sun in kilometers and miles, and how does this distance vary throughout the year due to the Earth's elliptical orbit?" 

This enhanced query not only asks for the average distance but also seeks additional context about the variation in distance, which can lead to more comprehensive information retrieval.
ranking docs
Answer: I don't know.


clasiffying query
retrieving analytical
sub queries for comprehensive analysis: ["What is the relationship between the Earth's distance from the Sun and the amount of solar radiation received?", "How do variations in the Earth's orbit influence seasonal climate patterns?", "What role does the Earth's distance from the Sun play in long-term climate changes, such as ice ages?", "How does the Earth's distance from the Sun compare to that of other 

This test block is where everything comes together. We are feeding in different types of questions—some factual, some analytical, others more subjective or dependent on context. The `AdaptiveRAG` system automatically runs each query through the classifier to determine its type and then uses the appropriate retrieval strategy underneath. It is a full end-to-end RAG engine that adapts to user intent, not just keywords.